# OpenAI Compatible API

SGLang provides an OpenAI compatible API for smooth transition from OpenAI services. Full reference of the API is available at [OpenAI API Reference](https://platform.openai.com/docs/api-reference).

This tutorial aims at these popular APIs:

- `chat/completions`
- `completions`
- `batches`
- `embeddings`(refer to [embedding_model.ipynb](embedding_model.ipynb))

## Chat Completions

### Usage

Similar to [send_request.ipynb](send_request.ipynb), we can send a chat completion request to SGLang server with OpenAI API format.

In [48]:
from IPython.display import display, HTML
from sglang.utils import lauch_sglang_server, wait_for_server, terminate_process

server_process = lauch_sglang_server(
    command="python -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct --port 30000 --host 0.0.0.0"
)

wait_for_server("http://localhost:30000")

display(HTML("<strong>Server is ready. Proceeding with the next steps.</strong>"))

/home/chenyang/miniconda3/envs/AlphaMeemory/lib/python3.11/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
[2024-10-28 00:43:20] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3.1-8B-Instruct', chat_template=None, is_embedding=False, host='0.0.0.0', port=30000, mem_fraction_static=0.88, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, tp_size=1, stream_interval=1, random_seed=779112371, constrained_json_whitespace_pattern=None, log_level='i

INFO 10-28 00:43:28 weight_utils.py:236] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  2.48it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:00<00:00,  2.32it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:00<00:00,  3.38it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  2.94it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  2.87it/s]

[2024-10-28 00:43:30 TP0] Load weight end. type=LlamaForCausalLM, dtype=torch.bfloat16, avail mem=32.22 GB
[2024-10-28 00:43:30 TP0] Memory pool end. avail mem=4.60 GB
[2024-10-28 00:43:30 TP0] Capture cuda graph begin. This can take up to several minutes.
[2024-10-28 00:43:37 TP0] max_total_num_tokens=217512, max_prefill_tokens=16384, max_running_requests=2049, context_len=131072
[2024-10-28 00:43:38] INFO:     Started server process [1156148]
[2024-10-28 00:43:38] INFO:     Waiting for appl

[2024-10-28 00:43:38] INFO:     127.0.0.1:52806 - "GET /v1/models HTTP/1.1" 200 OK


[2024-10-28 00:43:39] INFO:     127.0.0.1:52812 - "GET /get_model_info HTTP/1.1" 200 OK


[2024-10-28 00:43:39 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2024-10-28 00:43:39] INFO:     127.0.0.1:52820 - "POST /generate HTTP/1.1" 200 OK


[2024-10-28 00:43:39] The server is fired up and ready to roll!


In [38]:
import openai

# Always assign an api_key, even if not specified during server initialization.
# Setting an API key during server initialization is strongly recommended.

client = openai.Client(base_url="http://127.0.0.1:30000/v1", api_key="None")

# Chat completion example

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "system", "content": "You are a helpful AI assistant"},
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

display(HTML(f"<strong>Response: {response}</strong>"))

[2024-10-28 00:41:57 TP0] Prefill batch. #new-seq: 1, #new-token: 49, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2024-10-28 00:41:58] INFO:     127.0.0.1:35468 - "GET /get_model_info HTTP/1.1" 200 OK


[2024-10-28 00:41:58 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 1, cache hit rate: 1.79%, token usage: 0.00, #running-req: 1, #queue-req: 0
[2024-10-28 00:41:58] The server is fired up and ready to roll!


[2024-10-28 00:41:58] INFO:     127.0.0.1:35474 - "POST /generate HTTP/1.1" 200 OK
[2024-10-28 00:41:58] INFO:     127.0.0.1:35458 - "POST /v1/chat/completions HTTP/1.1" 200 OK


[2024-10-28 00:41:58 TP0] Decode batch. #running-req: 1, #token: 89, token usage: 0.00, gen throughput (token/s): 27.57, #queue-req: 0


### Parameters

The chat completions API accepts OpenAI Chat Completions API's parameters. Refer to [OpenAI Chat Completions API](https://platform.openai.com/docs/api-reference/chat/create) for more details.

Here is an example of a detailed chat completion request:

In [39]:
response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {
            "role": "system",
            "content": "You are a knowledgeable historian who provides concise responses.",
        },
        {"role": "user", "content": "Tell me about ancient Rome"},
        {
            "role": "assistant",
            "content": "Ancient Rome was a civilization centered in Italy.",
        },
        {"role": "user", "content": "What were their major achievements?"},
    ],
    temperature=0.3,  # Lower temperature for more focused responses
    max_tokens=100,  # Reasonable length for a concise response
    top_p=0.95,  # Slightly higher for better fluency
    stop=["\n\n"],  # Simple stop sequence
    presence_penalty=0.2,  # Mild penalty to avoid repetition
    frequency_penalty=0.2,  # Mild penalty for more natural language
    n=1,  # Single response is usually more stable
    seed=42,  # Keep for reproducibility
)

display(HTML(f"<strong>Response: {response}</strong>"))

[2024-10-28 00:41:58 TP0] Prefill batch. #new-seq: 1, #new-token: 48, #cached-token: 28, cache hit rate: 21.97%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2024-10-28 00:41:58] INFO:     127.0.0.1:35458 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Completions

### Usage

Completions API is similar to Chat Completions API, but without the `messages` parameter.

In [40]:
response = client.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    prompt="List 3 countries and their capitals.",
    temperature=0,
    max_tokens=64,
    n=1,
    stop=None,
)

display(HTML(f"<strong>Response: {response}</strong>"))

[2024-10-28 00:41:58 TP0] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 1, cache hit rate: 21.28%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-10-28 00:41:59 TP0] Decode batch. #running-req: 1, #token: 37, token usage: 0.00, gen throughput (token/s): 38.39, #queue-req: 0


[2024-10-28 00:42:00] INFO:     127.0.0.1:35458 - "POST /v1/completions HTTP/1.1" 200 OK


### Parameters

The completions API accepts OpenAI Completions API's parameters.  Refer to [OpenAI Completions API](https://platform.openai.com/docs/api-reference/completions/create) for more details.

Here is an example of a detailed completions request:

In [41]:
response = client.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    prompt="Write a short story about a space explorer.",
    temperature=0.7,  # Moderate temperature for creative writing
    max_tokens=150,  # Longer response for a story
    top_p=0.9,  # Balanced diversity in word choice
    stop=["\n\n", "THE END"],  # Multiple stop sequences
    presence_penalty=0.3,  # Encourage novel elements
    frequency_penalty=0.3,  # Reduce repetitive phrases
    n=1,  # Generate one completion
    seed=123,  # For reproducible results
)

display(HTML(f"<strong>Response: {response}</strong>"))

[2024-10-28 00:42:00 TP0] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 1, cache hit rate: 20.53%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-10-28 00:42:00 TP0] Decode batch. #running-req: 1, #token: 15, token usage: 0.00, gen throughput (token/s): 40.93, #queue-req: 0
[2024-10-28 00:42:01 TP0] Decode batch. #running-req: 1, #token: 55, token usage: 0.00, gen throughput (token/s): 42.20, #queue-req: 0
[2024-10-28 00:42:02 TP0] Decode batch. #running-req: 1, #token: 95, token usage: 0.00, gen throughput (token/s): 42.09, #queue-req: 0
[2024-10-28 00:42:03 TP0] Decode batch. #running-req: 1, #token: 135, token usage: 0.00, gen throughput (token/s): 41.94, #queue-req: 0


[2024-10-28 00:42:03] INFO:     127.0.0.1:35458 - "POST /v1/completions HTTP/1.1" 200 OK


## Batches

We have implemented the batches API for chat completions and completions. You can upload your requests in `jsonl` files, create a batch job, and retrieve the results when the batch job is completed (which takes longer but costs less).

The batches APIs are:

- `batches`
- `batches/{batch_id}/cancel`
- `batches/{batch_id}`

Here is an example of a batch job for chat completions, completions are similar.


In [42]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url="http://127.0.0.1:30000/v1", api_key="None")

requests = [
    {
        "custom_id": "request-1",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
            "messages": [
                {"role": "user", "content": "Tell me a joke about programming"}
            ],
            "max_tokens": 50,
        },
    },
    {
        "custom_id": "request-2",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
            "messages": [{"role": "user", "content": "What is Python?"}],
            "max_tokens": 50,
        },
    },
]

input_file_path = "batch_requests.jsonl"

with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    file_response = client.files.create(file=f, purpose="batch")

batch_response = client.batches.create(
    input_file_id=file_response.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

display(HTML(f"<strong>Batch job created with ID: {batch_response.id}</strong>"))

[2024-10-28 00:42:03] INFO:     127.0.0.1:35482 - "POST /v1/files HTTP/1.1" 200 OK
[2024-10-28 00:42:03] INFO:     127.0.0.1:35482 - "POST /v1/batches HTTP/1.1" 200 OK


[2024-10-28 00:42:03 TP0] Prefill batch. #new-seq: 2, #new-token: 30, #cached-token: 50, cache hit rate: 35.06%, token usage: 0.00, #running-req: 0, #queue-req: 0


In [43]:
while batch_response.status not in ["completed", "failed", "cancelled"]:
    time.sleep(3)
    print(f"Batch job status: {batch_response.status}...trying again in 3 seconds...")
    batch_response = client.batches.retrieve(batch_response.id)

if batch_response.status == "completed":
    print("Batch job completed successfully!")
    print(f"Request counts: {batch_response.request_counts}")

    result_file_id = batch_response.output_file_id
    file_response = client.files.content(result_file_id)
    result_content = file_response.read().decode("utf-8")

    results = [
        json.loads(line) for line in result_content.split("\n") if line.strip() != ""
    ]

    for result in results:
        display(HTML(f"<strong>Request {result['custom_id']}:</strong>"))
        display(HTML(f"<strong>Response: {result['response']}</strong>"))

    display(HTML("<strong>Cleaning up files...</strong>"))
    # Only delete the result file ID since file_response is just content
    client.files.delete(result_file_id)
else:
    display(
        HTML(f"<strong>Batch job failed with status: {batch_response.status}</strong>")
    )
    if hasattr(batch_response, "errors"):
        display(HTML(f"<strong>Errors: {batch_response.errors}</strong>"))

[2024-10-28 00:42:04 TP0] Decode batch. #running-req: 1, #token: 55, token usage: 0.00, gen throughput (token/s): 51.45, #queue-req: 0


Batch job status: validating...trying again in 3 seconds...
[2024-10-28 00:42:06] INFO:     127.0.0.1:35482 - "GET /v1/batches/batch_269765f7-9083-41db-8852-a6a1a2be930d HTTP/1.1" 200 OK
Batch job completed successfully!
Request counts: BatchRequestCounts(completed=2, failed=0, total=2)
[2024-10-28 00:42:06] INFO:     127.0.0.1:35482 - "GET /v1/files/backend_result_file-e13a36f9-c92c-4a7a-9814-435d93400f40/content HTTP/1.1" 200 OK


[2024-10-28 00:42:06] INFO:     127.0.0.1:35482 - "DELETE /v1/files/backend_result_file-e13a36f9-c92c-4a7a-9814-435d93400f40 HTTP/1.1" 200 OK


It takes a while to complete the batch job. You can use these two APIs to retrieve the batch job status or cancel the batch job.

1. `batches/{batch_id}`: Retrieve the batch job status.
2. `batches/{batch_id}/cancel`: Cancel the batch job.

Here is an example to check the batch job status.

In [49]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url="http://127.0.0.1:30000/v1", api_key="None")

requests = []
for i in range(100):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 500,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print(f"Created batch job with ID: {batch_job.id}")
print(f"Initial status: {batch_job.status}")

time.sleep(10)

max_checks = 5
for i in range(max_checks):
    batch_details = client.batches.retrieve(batch_id=batch_job.id)

    display(
        HTML(
            f"<strong>Batch job details (check {i+1} / {max_checks}) // ID: {batch_details.id} // Status: {batch_details.status} // Created at: {batch_details.created_at} // Input file ID: {batch_details.input_file_id} // Output file ID: {batch_details.output_file_id}</strong>"
        )
    )
    display(
        HTML(
            f"<strong>Request counts: Total: {batch_details.request_counts.total} // Completed: {batch_details.request_counts.completed} // Failed: {batch_details.request_counts.failed}</strong>"
        )
    )

    time.sleep(3)

[2024-10-28 00:43:52] INFO:     127.0.0.1:44608 - "POST /v1/files HTTP/1.1" 200 OK
[2024-10-28 00:43:52] INFO:     127.0.0.1:44608 - "POST /v1/batches HTTP/1.1" 200 OK
Created batch job with ID: batch_e558bbee-2dc2-4d0d-a317-492e80551aae
Initial status: validating


[2024-10-28 00:43:52 TP0] Prefill batch. #new-seq: 8, #new-token: 432, #cached-token: 8, cache hit rate: 1.79%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-10-28 00:43:52 TP0] Prefill batch. #new-seq: 92, #new-token: 2760, #cached-token: 2300, cache hit rate: 41.91%, token usage: 0.00, #running-req: 8, #queue-req: 0
[2024-10-28 00:43:53 TP0] Decode batch. #running-req: 100, #token: 6325, token usage: 0.03, gen throughput (token/s): 203.78, #queue-req: 0
[2024-10-28 00:43:55 TP0] Decode batch. #running-req: 100, #token: 10325, token usage: 0.05, gen throughput (token/s): 3476.48, #queue-req: 0
[2024-10-28 00:43:56 TP0] Decode batch. #running-req: 100, #token: 14325, token usage: 0.07, gen throughput (token/s): 3393.31, #queue-req: 0
[2024-10-28 00:43:57 TP0] Decode batch. #running-req: 100, #token: 18325, token usage: 0.08, gen throughput (token/s): 3309.35, #queue-req: 0
[2024-10-28 00:43:58 TP0] Decode batch. #running-req: 100, #token: 22325, token usage: 0.10, gen through

[2024-10-28 00:44:02] INFO:     127.0.0.1:48230 - "GET /v1/batches/batch_e558bbee-2dc2-4d0d-a317-492e80551aae HTTP/1.1" 200 OK


[2024-10-28 00:44:02 TP0] Decode batch. #running-req: 100, #token: 34325, token usage: 0.16, gen throughput (token/s): 3010.52, #queue-req: 0
[2024-10-28 00:44:03 TP0] Decode batch. #running-req: 100, #token: 38325, token usage: 0.18, gen throughput (token/s): 2943.53, #queue-req: 0


[2024-10-28 00:44:05] INFO:     127.0.0.1:48230 - "GET /v1/batches/batch_e558bbee-2dc2-4d0d-a317-492e80551aae HTTP/1.1" 200 OK


[2024-10-28 00:44:05 TP0] Decode batch. #running-req: 100, #token: 42325, token usage: 0.19, gen throughput (token/s): 2879.09, #queue-req: 0


[2024-10-28 00:44:06 TP0] Decode batch. #running-req: 100, #token: 46325, token usage: 0.21, gen throughput (token/s): 2817.34, #queue-req: 0
[2024-10-28 00:44:08 TP0] Decode batch. #running-req: 100, #token: 50325, token usage: 0.23, gen throughput (token/s): 2758.27, #queue-req: 0


[2024-10-28 00:44:08] INFO:     127.0.0.1:48230 - "GET /v1/batches/batch_e558bbee-2dc2-4d0d-a317-492e80551aae HTTP/1.1" 200 OK


[2024-10-28 00:44:11] INFO:     127.0.0.1:48230 - "GET /v1/batches/batch_e558bbee-2dc2-4d0d-a317-492e80551aae HTTP/1.1" 200 OK


[2024-10-28 00:44:14] INFO:     127.0.0.1:48230 - "GET /v1/batches/batch_e558bbee-2dc2-4d0d-a317-492e80551aae HTTP/1.1" 200 OK


Here is an example to cancel a batch job.

In [45]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url="http://127.0.0.1:30000/v1", api_key="None")

requests = []
for i in range(500):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 500,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

display(HTML(f"<strong>Created batch job with ID: {batch_job.id}</strong>"))
display(HTML(f"<strong>Initial status: {batch_job.status}</strong>"))

time.sleep(10)

try:
    cancelled_job = client.batches.cancel(batch_id=batch_job.id)
    display(
        HTML(f"<strong>Cancellation initiated. Status: {cancelled_job.status}</strong>")
    )
    assert cancelled_job.status == "cancelling"

    # Monitor the cancellation process
    while cancelled_job.status not in ["failed", "cancelled"]:
        time.sleep(3)
        cancelled_job = client.batches.retrieve(batch_job.id)
        display(HTML(f"<strong>Current status: {cancelled_job.status}</strong>"))

    # Verify final status
    assert cancelled_job.status == "cancelled"
    display(HTML("<strong>Batch job successfully cancelled</strong>"))

except Exception as e:
    display(HTML(f"<strong>Error during cancellation: {e}</strong>"))
    raise e

finally:
    try:
        del_response = client.files.delete(uploaded_file.id)
        if del_response.deleted:
            display(HTML("<strong>Successfully cleaned up input file</strong>"))
    except Exception as e:
        display(HTML(f"<strong>Error cleaning up: {e}</strong>"))
        raise e

[2024-10-28 00:42:31] INFO:     127.0.0.1:36362 - "POST /v1/files HTTP/1.1" 200 OK
[2024-10-28 00:42:31] INFO:     127.0.0.1:36362 - "POST /v1/batches HTTP/1.1" 200 OK


[2024-10-28 00:42:32 TP0] Prefill batch. #new-seq: 82, #new-token: 82, #cached-token: 4428, cache hit rate: 68.44%, token usage: 0.01, #running-req: 0, #queue-req: 0
[2024-10-28 00:42:32 TP0] Prefill batch. #new-seq: 291, #new-token: 8192, #cached-token: 7797, cache hit rate: 56.45%, token usage: 0.01, #running-req: 82, #queue-req: 127
[2024-10-28 00:42:33 TP0] Prefill batch. #new-seq: 128, #new-token: 3826, #cached-token: 3214, cache hit rate: 54.16%, token usage: 0.05, #running-req: 372, #queue-req: 1
[2024-10-28 00:42:36 TP0] Decode batch. #running-req: 500, #token: 29025, token usage: 0.13, gen throughput (token/s): 1164.09, #queue-req: 0
[2024-10-28 00:42:39 TP0] Decode batch. #running-req: 500, #token: 49025, token usage: 0.23, gen throughput (token/s): 5589.00, #queue-req: 0


[2024-10-28 00:42:41] INFO:     127.0.0.1:45764 - "POST /v1/batches/batch_5d87043a-8b49-4cd1-8bf5-1a42a99859e3/cancel HTTP/1.1" 200 OK


[2024-10-28 00:42:44] INFO:     127.0.0.1:45764 - "GET /v1/batches/batch_5d87043a-8b49-4cd1-8bf5-1a42a99859e3 HTTP/1.1" 200 OK


[2024-10-28 00:42:44] INFO:     127.0.0.1:45764 - "DELETE /v1/files/backend_input_file-954d76cd-e92f-4bf9-93d0-6561587e7fdb HTTP/1.1" 200 OK


In [46]:
terminate_process(server_process)

[2024-10-28 00:42:44] INFO:     Shutting down
[2024-10-28 00:42:45] INFO:     Waiting for application shutdown.
[2024-10-28 00:42:45] INFO:     Application shutdown complete.
[2024-10-28 00:42:45] INFO:     Finished server process [1154958]
W1028 00:42:45.164000 140548813764160 torch/_inductor/compile_worker/subproc_pool.py:126] SubprocPool unclean exit
